Exploration of ETL

In [ ]:
for date, neo_list in neo_data.items():
    for neo in neo_list:
        cursor.execute('''
                    INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
                    ''', (
                        neo['id'],
                        neo['name'],
                        neo['is_potentially_hazardous_asteroid'],
                        neo['is_sentry_object'],
                        neo['close_approach_data'],
                        neo['estimated_diameter']
                    ))
conn.commit()

In [7]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")

link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

        DROP TABLE IF EXISTS neo;

        CREATE TABLE neo (
                        id text,
                        data json
        )
    ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                        (neo['id'], json.dumps(neo)))
    conn.commit()

if __name__ == '__main__':
    load_data()

df = pd.read_sql_query('''SELECT data FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

df = pd.concat([json_df, df], axis=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 18 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    83 non-null     object 
 1   neo_reference_id                                      83 non-null     object 
 2   name                                                  83 non-null     object 
 3   nasa_jpl_url                                          83 non-null     object 
 4   absolute_magnitude_h                                  83 non-null     float64
 5   is_potentially_hazardous_asteroid                     83 non-null     bool   
 6   close_approach_data                                   83 non-null     object 
 7   is_sentry_object                                      83 non-null     bool   
 8   links.self                                            83 non-n

In [10]:
df_clean = df.drop(['neo_reference_id', 'links.self', 
         'estimated_diameter.kilometers.estimated_diameter_min',
         'estimated_diameter.kilometers.estimated_diameter_max',
         'estimated_diameter.miles.estimated_diameter_min',
         'estimated_diameter.miles.estimated_diameter_max',
         'estimated_diameter.feet.estimated_diameter_min',
         'estimated_diameter.feet.estimated_diameter_max'], 
         axis=1)

df_clean

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,data
0,2226554,226554 (2003 WR21),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2226554,19.640,True,"[{'close_approach_date': '2023-09-19', 'close_...",False,313.729225,701.519874,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2470510,470510 (2008 CJ116),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2470510,19.220,False,"[{'close_approach_date': '2023-09-19', 'close_...",False,380.675544,851.216393,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3023157,(1999 TY2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3023157,23.300,False,"[{'close_approach_date': '2023-09-19', 'close_...",False,58.150704,130.028927,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3520762,(2010 JM41),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3520762,21.050,False,"[{'close_approach_date': '2023-09-19', 'close_...",False,163.890951,366.471308,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3579748,(2011 SY120),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3579748,23.200,False,"[{'close_approach_date': '2023-09-19', 'close_...",False,60.891262,136.157002,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...
78,54202343,(2021 TQ1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54202343,28.090,False,"[{'close_approach_date': '2023-09-21', 'close_...",False,6.405529,14.323197,{'links': {'self': 'http://api.nasa.gov/neo/re...
79,54244535,(2022 CT),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54244535,25.730,False,"[{'close_approach_date': '2023-09-21', 'close_...",False,18.991312,42.465865,{'links': {'self': 'http://api.nasa.gov/neo/re...
80,54310136,(2022 SD29),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54310136,26.480,False,"[{'close_approach_date': '2023-09-21', 'close_...",False,13.444820,30.063530,{'links': {'self': 'http://api.nasa.gov/neo/re...
81,54359399,(2023 KK1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54359399,20.200,True,"[{'close_approach_date': '2023-09-21', 'close_...",False,242.412481,542.050786,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [29]:
close_approach_data = pd.json_normalize(df_clean['close_approach_data'])

close_approach_data

,0
0,"{'close_approach_date': '2023-09-19', 'close_a..."
1,"{'close_approach_date': '2023-09-19', 'close_a..."
2,"{'close_approach_date': '2023-09-19', 'close_a..."
3,"{'close_approach_date': '2023-09-19', 'close_a..."
4,"{'close_approach_date': '2023-09-19', 'close_a..."
...,...
78,"{'close_approach_date': '2023-09-21', 'close_a..."
79,"{'close_approach_date': '2023-09-21', 'close_a..."
80,"{'close_approach_date': '2023-09-21', 'close_a..."
81,"{'close_approach_date': '2023-09-21', 'close_a..."
